In [1]:
#tianjianmeng ml212jt

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import seaborn as sns
import torch
import matplotlib.pyplot as plt
from nltk import FreqDist
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
import itertools
import re
import nltk
import time
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
import networkx as nx
from transformers import BartTokenizer, BartForConditionalGeneration
import spacy
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader as api
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

### Web scraping

In [3]:
html_my = """<tbody>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=ProfilePic.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=ProfilePic.jpg&amp;jpegquality=80" alt="Dr. Noorhan Abbas">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8807/dr-noorhan-abbas">Abbas</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Noorhan</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 0084</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:N.H.Abbas@leeds.ac.uk">N.H.Abbas@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research &amp; Teaching Fellow </span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/13233/professor-isolde-adler">Adler</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Isolde</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 6819</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:I.M.Adler@leeds.ac.uk">I.M.Adler@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Visiting Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Untitled_design__3__4.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Untitled_design__3__4.png&amp;jpegquality=80" alt="Dr Ban Al-Jassani">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/mechanical-engineering/staff/12626/dr-ban-al-jassani">Al-Jassani</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Ban Adil Naji</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:B.A.N.Al-Jassani@leeds.ac.uk">B.A.N.Al-Jassani@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow/Teaching Assistant </span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Bogdan_Alecu_photo_square_400.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Bogdan_Alecu_photo_square_400.jpg&amp;jpegquality=80" alt="Dr Bogdan Alecu">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/10225/dr-bogdan-alecu">Alecu</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Bogdan</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:B.Alecu@leeds.ac.uk">B.Alecu@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=profile_4.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=profile_4.png&amp;jpegquality=80" alt="Dr Sharib Ali">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/11465/dr-sharib-ali">Ali</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Sharib</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:S.S.Ali@leeds.ac.uk">S.S.Ali@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=alsalka-mhd-ammar.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=alsalka-mhd-ammar.jpg&amp;jpegquality=80" alt="Dr Mohammad Ammar Alsalka">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/635/dr-mohammad-ammar-alsalka">Alsalka</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Mhd Ammar</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 1142</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.A.Alsalka@leeds.ac.uk">M.A.Alsalka@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer in Computing</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=AAltahhan_3.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=AAltahhan_3.jpg&amp;jpegquality=80" alt="Dr Abdulrahman Altahhan">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8784/dr-abdulrahman-altahhan">Altahhan</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Abdulrahman</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 1310</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.Altahhan@leeds.ac.uk">A.Altahhan@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Senior Teaching Fellow in Artificial Intelligence </span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=atwell-eric.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=atwell-eric.jpg&amp;jpegquality=80" alt="Professor Eric Atwell">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/33/professor-eric-atwell">Atwell</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Eric</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5761</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:E.S.Atwell@leeds.ac.uk">E.S.Atwell@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Artificial Intelligence for Language</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Untitled_design__70__1.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Untitled_design__70__1.png&amp;jpegquality=80" alt="Dr Zeeshan">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/12946/dr-zeeshan">Babar</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Muhammad Zeeshan</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.Z.Babar@leeds.ac.uk">M.Z.Babar@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Teaching Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=1638730846808.jpeg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=1638730846808.jpeg&amp;jpegquality=80" alt="Dr Fatemeh Banaie">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/10151/dr-fatemeh-banaie">Banaie Heravan</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Fatemeh</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:F.BanaieHeravan@leeds.ac.uk">F.BanaieHeravan@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=lowe-amy.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=lowe-amy.jpg&amp;jpegquality=80" alt="Dr Amy Lowe">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/6763/dr-amy-lowe">Beloe</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Amy</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.Beloe@leeds.ac.uk">A.Beloe@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Teaching Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=brandonbennett.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=brandonbennett.jpg&amp;jpegquality=80" alt="Dr Brandon Bennett">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/121/dr-brandon-bennett">Bennett</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Brandon</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 1070</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:B.Bennett@leeds.ac.uk">B.Bennett@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=WIN_20220530_13_01_33_Pro.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=WIN_20220530_13_01_33_Pro.jpg&amp;jpegquality=80" alt="Harriet Bennett">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/faculty-engineering-physical-sciences/staff/13223/harriet-bennett">Bennett</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Harriet</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:H.G.Bennett@leeds.ac.uk">H.G.Bennett@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Student Education Service Manager</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=image_250x302x.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=image_250x302x.png&amp;jpegquality=80" alt="Dr. Markus Billeter">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8810/dr-markus-billeter">Billeter</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Markus</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.Billeter@leeds.ac.uk">M.Billeter@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Assistant Professor</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Peter_Bollada.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Peter_Bollada.jpg&amp;jpegquality=80" alt="Dr Peter Bollada">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/523/dr-peter-bollada">Bollada</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Peter</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 2567</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:P.C.Bollada@leeds.ac.uk">P.C.Bollada@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/10418/amy-brereton">Brereton</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Amy</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.M.Brereton@leeds.ac.uk">A.M.Brereton@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Teaching Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/686/ruksana-bukhari-bibi">Bukhari-Bibi</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Ruksana</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 6788</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:R.Bukhari-Bibi@leeds.ac.uk">R.Bukhari-Bibi@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Education Service Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=bulpitt-andy.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=bulpitt-andy.jpg&amp;jpegquality=80" alt="Professor Andy Bulpitt">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/115/professor-andy-bulpitt">Bulpitt</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Andy</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 6816</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.J.Bulpitt@leeds.ac.uk">A.J.Bulpitt@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Computer Science</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/11489/martin-callaghan">Callaghan</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Martin</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 0273</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.Callaghan@leeds.ac.uk">M.Callaghan@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=carr-hamish.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=carr-hamish.jpg&amp;jpegquality=80" alt="Professor Hamish Carr">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/499/professor-hamish-carr">Carr</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Hamish</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 7042</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:H.Carr@leeds.ac.uk">H.Carr@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Computer Science</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=514ae3c1e3b40f87ab59a340c2ddd2b0.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=514ae3c1e3b40f87ab59a340c2ddd2b0.jpg&amp;jpegquality=80" alt="Dr Natasha Shakhlevich">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/285/dr-natasha-shakhlevich">Chakhlevitch</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Natasha Shakhlevich</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5444</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:N.Shakhlevich@leeds.ac.uk">N.Shakhlevich@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Senior Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/12765/omar-shaur-choudhry">Choudhry</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Omar</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:O.Choudhry@leeds.ac.uk">O.Choudhry@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Teaching Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=thumbnail_20220429_101925.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=thumbnail_20220429_101925.jpg&amp;jpegquality=80" alt="Deborah Clarke">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/11490/deborah-clarke">Clarke</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Deborah</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:D.Clarke2@leeds.ac.uk">D.Clarke2@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Administrative Support Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=e8c55345a06c2c397c31c5eabb95c8fa.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=e8c55345a06c2c397c31c5eabb95c8fa.jpg&amp;jpegquality=80" alt="Professor Netta Cohen">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/301/professor-netta-cohen">Cohen</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Netta</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 6789</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:N.Cohen@leeds.ac.uk">N.Cohen@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Tony_Cohn.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Tony_Cohn.jpg&amp;jpegquality=80" alt="Professor Anthony (Tony) G Cohn, FREng, CEng, CITP">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/76/professor-anthony-tony-g-cohn-freng-ceng-citp">Cohn</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Anthony</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5482</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.G.Cohn@leeds.ac.uk">A.G.Cohn@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Automated Reasoning</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=profile_3.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=profile_3.png&amp;jpegquality=80" alt="Mr. Minh Thang Dang">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/10328/mr-minh-thang-dang">Dang</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Minh</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.T.Dang@leeds.ac.uk">M.T.Dang@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Teaching Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Marc_Dekamps.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Marc_Dekamps.jpg&amp;jpegquality=80" alt="Dr Marc de Kamps">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/390/dr-marc-de-kamps">de Kamps</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Marc</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5322</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.deKamps@leeds.ac.uk">M.deKamps@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Associate Professor</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8419/sara">Dickinson</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Sara</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 7049</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:S.D.J.Dickinson@leeds.ac.uk">S.D.J.Dickinson@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Administrative Support Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Vania_Dimitrova.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Vania_Dimitrova.jpg&amp;jpegquality=80" alt="Professor Vania Dimitrova">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/184/professor-vania-dimitrova">Dimitrova</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Vania</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 1674</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:V.G.Dimitrova@leeds.ac.uk">V.G.Dimitrova@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Human-Centred Artificial Intelligence</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=djemame-karim.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=djemame-karim.jpg&amp;jpegquality=80" alt="Professor Karim Djemame">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/187/professor-karim-djemame">Djemame</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Karim</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 6590</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:K.Djemame@leeds.ac.uk">K.Djemame@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Mehmet_Dogar.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Mehmet_Dogar.jpg&amp;jpegquality=80" alt="Dr Mehmet Dogar">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/743/dr-mehmet-dogar">Dogar</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Mehmet</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5777</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.R.Dogar@leeds.ac.uk">M.R.Dogar@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Associate Professor</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=David_Duke_1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=David_Duke_1.jpg&amp;jpegquality=80" alt="Professor David Duke">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/334/professor-david-duke">Duke</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">David</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 6800</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:D.J.Duke@leeds.ac.uk">D.J.Duke@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor Emeritus of Computer Science</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Martin_Dyer_2.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Martin_Dyer_2.jpg&amp;jpegquality=80" alt="Professor Martin Dyer">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/52/professor-martin-dyer">Dyer</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Martin</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5442</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.E.Dyer@leeds.ac.uk">M.E.Dyer@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=76098627e62aba735fb54d64cebf8235.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=76098627e62aba735fb54d64cebf8235.jpg&amp;jpegquality=80" alt="Dr Nick Efford">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/92/dr-nick-efford">Efford</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Nick</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 6809</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:N.D.Efford@leeds.ac.uk">N.D.Efford@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Senior Teaching Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Untitled_design__79__1.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Untitled_design__79__1.png&amp;jpegquality=80" alt="Dr Samson Fabiyi">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/9808/dr-samson-fabiyi">Fabiyi</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Samson</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 2463</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:S.D.Fabiyi@leeds.ac.uk">S.D.Fabiyi@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Transnational Teaching Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=8f6e158936e73a2a2f4639d2d2cae506.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=8f6e158936e73a2a2f4639d2d2cae506.jpg&amp;jpegquality=80" alt="Charlotte Francis">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/321/charlotte-francis">Francis</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Charlotte</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5821</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:C.Francis@leeds.ac.uk">C.Francis@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Assistant School Education Service Mger</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=FrangiMini2016SQ.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=FrangiMini2016SQ.png&amp;jpegquality=80" alt="Professor Alejandro F Frangi">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/1535/professor-alejandro-f-frangi">Frangi</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Alex</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 9640</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.Frangi@leeds.ac.uk">A.Frangi@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Diamond Jubilee Chair in Computational Medicine | Royal Academy </span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/faculty-engineering-physical-sciences/staff/6424/dr-helen-freeman">Freeman</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Helen</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 9071</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:H.M.Freeman@leeds.ac.uk">H.M.Freeman@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Network Manager</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Joseph.JPG&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Joseph.JPG&amp;jpegquality=80" alt="Joseph Ghaffari Motlagh">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8191/joseph-ghaffari-motlagh">Ghaffari Motlagh</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Yousef</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 3449</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:Y.GhaffariMotlagh@leeds.ac.uk">Y.GhaffariMotlagh@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Visiting Academic</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8344/sunjeev-ghir">Ghir</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Sunjeev</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 1650</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:S.Ghir@leeds.ac.uk">S.Ghir@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">TBC</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/1837/shokoufeh-golshani">Golshani</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Shokoufeh</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:S.Golshani@leeds.ac.uk">S.Golshani@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Marie Curie Researcher</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=8d68a96aa6dc6bdea3056b634b2b7dd1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=8d68a96aa6dc6bdea3056b634b2b7dd1.jpg&amp;jpegquality=80" alt="Anne Hayler">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/332/anne-hayler">Hayler</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Anne</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 2228</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.Hayler@leeds.ac.uk">A.Hayler@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">School Education Service Manager</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=hazlehurst-thomas.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=hazlehurst-thomas.jpg&amp;jpegquality=80" alt="Dr Tom Hazlehurst">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/chemical-engineering/staff/835/dr-tom-hazlehurst">Hazlehurst</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Thomas</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:T.Hazlehurst@leeds.ac.uk">T.Hazlehurst@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=David_Head.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=David_Head.jpg&amp;jpegquality=80" alt="Dr David Head">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/542/dr-david-head">Head</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">David</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 4693</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:D.Head@leeds.ac.uk">D.Head@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/283/beth-hilditch">Hilditch</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Beth</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5465</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:B.A.Hilditch@leeds.ac.uk">B.A.Hilditch@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Education Service Officer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=IMG_3474_window.JPG&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=IMG_3474_window.JPG&amp;jpegquality=80" alt="Professor David Hogg">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/84/professor-david-hogg">Hogg</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">David</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5765</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:D.C.Hogg@leeds.ac.uk">D.C.Hogg@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Artificial Intelligence</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Kattia_4.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Kattia_4.png&amp;jpegquality=80" alt="Mrs. Kattia Lisette Hoz de Vila">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/6749/mrs-kattia-lisette-hoz-de-vila">Hoz de Vila Eduardo</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Kattia</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:K.HozdeVila@leeds.ac.uk">K.HozdeVila@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Software Engineer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=image823.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=image823.png&amp;jpegquality=80" alt="Dr Yanlong Huang">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8178/dr-yanlong-huang">Huang</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Yanlong</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 3505</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:Y.L.Huang@leeds.ac.uk">Y.L.Huang@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">University Academic Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Jimack_1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Jimack_1.jpg&amp;jpegquality=80" alt="Professor Peter Jimack">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/82/professor-peter-jimack">Jimack</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Peter</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 2002</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:P.K.Jimack@leeds.ac.uk">P.K.Jimack@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Scientific Computing</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Owen_Johnson_2.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Owen_Johnson_2.jpg&amp;jpegquality=80" alt="Owen Johnson">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/259/owen-johnson">Johnson</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Owen</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5459</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:O.A.Johnson@leeds.ac.uk">O.A.Johnson@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Senior Teaching Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Untitled_design__94_.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Untitled_design__94_.png&amp;jpegquality=80" alt="Dr Rafael Kuffner dos Anjos">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/9770/dr-rafael-kuffner-dos-anjos">Kuffner Dos Anjos</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Rafael</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 3625</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:R.KuffnerdosAnjos@leeds.ac.uk">R.KuffnerdosAnjos@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer in Computer Graphics</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=satish_2.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=satish_2.jpg&amp;jpegquality=80" alt="Dr Satish Kumar">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/9634/dr-satish-kumar">Kumar</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Satish</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 3722</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:S.Kumar3@leeds.ac.uk">S.Kumar3@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=d31c04b9b18f187adf52203ce1b97155.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=d31c04b9b18f187adf52203ce1b97155.jpg&amp;jpegquality=80" alt="Professor Raymond Kwan">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/37/professor-raymond-kwan">Kwan</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Raymond</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5760</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:R.S.Kwan@leeds.ac.uk">R.S.Kwan@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Scheduling</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=newphoto_1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=newphoto_1.jpg&amp;jpegquality=80" alt="Dr Toni Lassila">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/1542/dr-toni-lassila">Lassila</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Toni</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 3724</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:T.Lassila@leeds.ac.uk">T.Lassila@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=lau3_1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=lau3_1.jpg&amp;jpegquality=80" alt="Kelvin Lau">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8195/kelvin-lau">Lau</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Hui</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 3748</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:H.K.Lau@leeds.ac.uk">H.K.Lau@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Teaching Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/10007/dr-wiktoria-lawniczak">Lawniczak</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Wiktoria</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:W.Lawniczak@leeds.ac.uk">W.Lawniczak@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">KTP Associate – Senior Data Scientist</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=ale_passport_uk___400X400.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=ale_passport_uk___400X400.jpg&amp;jpegquality=80" alt="Mr. Alejandro Lebrero">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/6758/mr-alejandro-lebrero">Lebrero</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Alejandro</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.Lebrero@leeds.ac.uk">A.Lebrero@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Software Engineer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=leng-joanna.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=leng-joanna.jpg&amp;jpegquality=80" alt="Joanna Leng">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/1509/joanna-leng">Leng</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Joanna</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 3809</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:J.Leng@leeds.ac.uk">J.Leng@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Senior Research Software Engineering Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('');">
                    
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/771/dr-matteo-leonetti">Leonetti</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Matteo</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5792</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.Leonetti@leeds.ac.uk">M.Leonetti@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=JianLiu.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=JianLiu.jpg&amp;jpegquality=80" alt="Dr Jian Liu">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/9216/dr-jian-liu">Liu</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Jian</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:J.Liu9@leeds.ac.uk">J.Liu9@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/faculty-engineering-physical-sciences/staff/11398/mr-lukasz-liwski">Liwski</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Lukasz</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:L.Liwski@leeds.ac.uk">L.Liwski@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Student Education Service Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Derek_Masee.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Derek_Masee.jpg&amp;jpegquality=80" alt="Derek Magee">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/172/derek-magee">Magee</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Derek</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 6819</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:D.R.Magee@leeds.ac.uk">D.R.Magee@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=perfil_franco.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=perfil_franco.jpg&amp;jpegquality=80" alt="Franco Matzkin">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/11493/franco-matzkin">Matzkin</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Victor</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:V.F.Matzkin@leeds.ac.uk">V.F.Matzkin@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">PhD Student</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Dr_Mantas_Mikaitis.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Dr_Mantas_Mikaitis.png&amp;jpegquality=80" alt="Dr Mantas Mikaitis">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/12197/dr-mantas-mikaitis">Mikaitis</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Mantas</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.Mikaitis@leeds.ac.uk">M.Mikaitis@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=muller-haiko.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=muller-haiko.jpg&amp;jpegquality=80" alt="Dr Haiko Muller">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/241/dr-haiko-muller">Muller</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Haiko</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5445</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:H.Muller@leeds.ac.uk">H.Muller@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Senior Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/11922/dr-zeinab-nezami">Nezami</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Zeinab</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:Z.Nezami@leeds.ac.uk">Z.Nezami@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/chemical-engineering/staff/6598/dr-thai-thu-hien-nguyen">Nguyen</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Hien</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:H.Nguyen1@leeds.ac.uk">H.Nguyen1@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8268/thanh-nguyen">Nguyen</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Thanh-Luan</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 4315</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:T.L.Nguyen@leeds.ac.uk">T.L.Nguyen@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">TBC</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/electronic-engineering/staff/6843/dr-luzhen-nie">Nie</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Luzhen</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:L.Nie@leeds.ac.uk">L.Nie@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Sebastian_ordyniak.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Sebastian_ordyniak.jpg&amp;jpegquality=80" alt="Dr. Sebastian Ordyniak">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8786/dr-sebastian-ordyniak">Ordyniak</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Sebastian</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:S.Ordyniak@leeds.ac.uk">S.Ordyniak@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer (Algorithms and Complexity)</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/10329/dr-giacomo-paesani">Paesani</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Giacomo</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:G.Paesani@leeds.ac.uk">G.Paesani@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow (Algorithms and Complexity)</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=IMG_6610.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=IMG_6610.png&amp;jpegquality=80" alt="Dr Rafael Papallas">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/6769/dr-rafael-papallas">Papallas</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Rafael</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:R.Papallas@leeds.ac.uk">R.Papallas@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=a27baf9e9e032626ba19c085393ce68c.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=a27baf9e9e032626ba19c085393ce68c.jpg&amp;jpegquality=80" alt="Dr Jonathan Pickering">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/540/dr-jonathan-pickering">Pickering</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Jonathan</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5836</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:J.H.Pickering@leeds.ac.uk">J.H.Pickering@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=photo5_1.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=photo5_1.png&amp;jpegquality=80" alt="Dr Evangelos Pournaras">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/6446/dr-evangelos-pournaras">Pournaras</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Evangelos</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:E.Pournaras@leeds.ac.uk">E.Pournaras@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Associate Professor</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=44697006_1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=44697006_1.jpg&amp;jpegquality=80" alt="Dr. Arash Rabbani">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/11422/dr-arash-rabbani">Rabbani</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Arash</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.Rabbani@leeds.ac.uk">A.Rabbani@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer (Assistant Professor)</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=AL4_3530_1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=AL4_3530_1.jpg&amp;jpegquality=80" alt="Dr Thomas Ranner">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/643/dr-thomas-ranner">Ranner</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Thomas</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 4697</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:T.Ranner@leeds.ac.uk">T.Ranner@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Nishant_Ravikumar.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Nishant_Ravikumar.jpg&amp;jpegquality=80" alt="Dr Nishant Ravikumar">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/1846/dr-nishant-ravikumar">Ravikumar</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Nishant</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:N.Ravikumar@leeds.ac.uk">N.Ravikumar@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer in Computer Science</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/maths/staff/12381/meg-richards">Richards</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Megan</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.K.Richards@leeds.ac.uk">M.K.Richards@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Demonstrator/Module Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8787/dr-thomas-richardson">Richardson</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Thomas</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:T.Richardson1@leeds.ac.uk">T.Richardson1@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Teaching Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=e9d4d4a1516f5a1ac703735f55fa7df7.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=e9d4d4a1516f5a1ac703735f55fa7df7.jpg&amp;jpegquality=80" alt="Professor Roy Ruddle">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/257/professor-roy-ruddle">Ruddle</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Roy</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 1711</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:R.A.Ruddle@leeds.ac.uk">R.A.Ruddle@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Computing</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/9772/hamidreza-saligheh-rad">Saligheh Rad</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Hamidreza</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:H.SalighehRad@leeds.ac.uk">H.SalighehRad@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">TBC</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=lDzjBFOo_400x400.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=lDzjBFOo_400x400.jpg&amp;jpegquality=80" alt="Dr. Duygu Sarikaya">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/13320/dr-duygu-sarikaya">Sarikaya</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Duygu</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:D.Sarikaya@leeds.ac.uk">D.Sarikaya@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer in Computer Science</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=107_1.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=107_1.png&amp;jpegquality=80" alt="Dr Ali Sarrami-Foroushani">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/6780/dr-ali-sarrami-foroushani">Sarrami Foroushani</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Ali</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.Sarrami@leeds.ac.uk">A.Sarrami@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=2c0af244528dcf4bf817f6c7d9e3919d.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=2c0af244528dcf4bf817f6c7d9e3919d.jpg&amp;jpegquality=80" alt="Claire Savy">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/621/claire-savy">Savy</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Claire</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5449</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:C.Savy@leeds.ac.uk">C.Savy@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Centre Manager</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Steve.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Steve.jpg&amp;jpegquality=80" alt="Professor Steve Scott">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/faculty-engineering-physical-sciences/staff/4601/professor-steve-scott">Scott</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Stephen</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:S.K.Scott@leeds.ac.uk">S.K.Scott@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Head of School</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=IMG_20221231_201248_008__1_.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=IMG_20221231_201248_008__1_.jpg&amp;jpegquality=80" alt="Abhinav Sharma">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/12771/abhinav-sharma">Sharma</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Abhinav</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.Sharma1@leeds.ac.uk">A.Sharma1@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Software Engineer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/6782/giulia-sindoni">Sindoni</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Giulia</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 4821</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:G.Sindoni@leeds.ac.uk">G.Sindoni@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Demonstrator/Module Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=f4f95c22c21ebd9763c6e9be275aa6d1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=f4f95c22c21ebd9763c6e9be275aa6d1.jpg&amp;jpegquality=80" alt="Dr John Stell">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/258/dr-john-stell">Stell</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">John</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 1076</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:J.G.Stell@leeds.ac.uk">J.G.Stell@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Senior Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=kristina.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=kristina.jpg&amp;jpegquality=80" alt="Professor Kristina Vušković">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/249/professor-kristina-vuskovic">Vuskovic</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Kristina</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5443</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:K.Vuskovic@leeds.ac.uk">K.Vuskovic@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Algorithms and Combinatorics</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=46d6e703fb7c08526d97393060bc23da.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=46d6e703fb7c08526d97393060bc23da.jpg&amp;jpegquality=80" alt="Dr Mark Walkley">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/189/dr-mark-walkley">Walkley</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Mark</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5684</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:M.A.Walkley@leeds.ac.uk">M.A.Walkley@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=He_Wang.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=He_Wang.jpg&amp;jpegquality=80" alt="Dr He Wang">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/868/dr-he-wang">Wang</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">He</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5767</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:H.E.Wang@leeds.ac.uk">H.E.Wang@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Associate Professor</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=photo2_1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=photo2_1.jpg&amp;jpegquality=80" alt="Dr Yongxing Wang">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/faculty-engineering-physical-sciences/staff/1600/dr-yongxing-wang">Wang</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Yongxing</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 4874</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:Y.Wang3@leeds.ac.uk">Y.Wang3@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=portrait_1.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=portrait_1.jpg&amp;jpegquality=80" alt="Professor Zheng Wang">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/6452/professor-zheng-wang">Wang</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Zheng</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 1077</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:Z.Wang5@leeds.ac.uk">Z.Wang5@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Intelligent Software Technology</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/11423/xiao-wang">Wang</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Xiao</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:X.Wang16@leeds.ac.uk">X.Wang16@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Teaching Assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=99b9545809e957a4a544ef9d2202a1bd.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=99b9545809e957a4a544ef9d2202a1bd.jpg&amp;jpegquality=80" alt="Dr Lijun Wei">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/760/dr-lijun-wei">Wei</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Lijun</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:L.J.Wei@leeds.ac.uk">L.J.Wei@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/9416/adam-wilkinson">Wilkinson</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Adam</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.J.Wilkinson@leeds.ac.uk">A.J.Wilkinson@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Student Education Service Assistant Manager</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=wilson-samuel.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=wilson-samuel.jpg&amp;jpegquality=80" alt="Dr Samuel Wilson">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/573/dr-samuel-wilson">Wilson</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Samuel</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5474</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:S.S.Wilson@leeds.ac.uk">S.S.Wilson@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Lecturer of Computing</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8269/dr-xia">Xia</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Yan</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:Y.Xia@leeds.ac.uk">Y.Xia@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Jie_Xu.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Jie_Xu.jpg&amp;jpegquality=80" alt="Professor Jie Xu">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/331/professor-jie-xu">Xu</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Jie</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5193</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:J.Xu@leeds.ac.uk">J.Xu@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Professor of Computing</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=FB_IMG_1541828614755.jpg&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=FB_IMG_1541828614755.jpg&amp;jpegquality=80" alt="Dr. Omer Yuval">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8789/dr-omer-yuval">Yuval</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Omer</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content"></span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:O.Yuval@leeds.ac.uk">O.Yuval@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research assistant</span></td>
            </tr>
                          <tr>
                <td><b class="tablesaw-cell-label"></b> <span class="tablesaw-cell-content">
                                                          <span class="rs-img img-avatar img-avatar-sm" style="background-image: url('//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Arezoo_1.png&amp;jpegquality=80');">
                    <img src="//eps.leeds.ac.uk/site/custom_scripts/image.php?w=60&amp;h=60&amp;filename=Arezoo_1.png&amp;jpegquality=80" alt="Dr. Arezoo Zakeri">
                    </span>
                                </span></td>
                <td class="title"><b class="tablesaw-cell-label">Last name</b> <span class="tablesaw-cell-content"><a href="https://eps.leeds.ac.uk/computing/staff/8199/dr-arezoo-zakeri">Zakeri</a></span></td>
                <td><b class="tablesaw-cell-label">First name</b> <span class="tablesaw-cell-content">Arezoo</span></td>
                <td class="no-wrap"><b class="tablesaw-cell-label">Phone number</b> <span class="tablesaw-cell-content">+44(0)113 343 5254</span></td>
                <!-- <td>  </td> -->
                <td class="no-wrap"><b class="tablesaw-cell-label">Email</b> <span class="tablesaw-cell-content"><a href="mailto:A.Zakeri@leeds.ac.uk">A.Zakeri@leeds.ac.uk</a></span></td>
                <td><b class="tablesaw-cell-label">Position</b> <span class="tablesaw-cell-content">Research Fellow</span></td>
            </tr>
                    </tbody>"""
url = []

soup = BeautifulSoup(html_my, 'html.parser')  # 创建BeautifulSoup对象

for link in soup.find_all('a'):
    href = link.get('href')
    if href.startswith('https'):  # 只输出以https开头的链接
        url.append(href)

In [32]:
print(url)

['https://eps.leeds.ac.uk/computing/staff/8807/dr-noorhan-abbas', 'https://eps.leeds.ac.uk/computing/staff/13233/professor-isolde-adler', 'https://eps.leeds.ac.uk/mechanical-engineering/staff/12626/dr-ban-al-jassani', 'https://eps.leeds.ac.uk/computing/staff/10225/dr-bogdan-alecu', 'https://eps.leeds.ac.uk/computing/staff/11465/dr-sharib-ali', 'https://eps.leeds.ac.uk/computing/staff/635/dr-mohammad-ammar-alsalka', 'https://eps.leeds.ac.uk/computing/staff/8784/dr-abdulrahman-altahhan', 'https://eps.leeds.ac.uk/computing/staff/33/professor-eric-atwell', 'https://eps.leeds.ac.uk/computing/staff/12946/dr-zeeshan', 'https://eps.leeds.ac.uk/computing/staff/10151/dr-fatemeh-banaie', 'https://eps.leeds.ac.uk/computing/staff/6763/dr-amy-lowe', 'https://eps.leeds.ac.uk/computing/staff/121/dr-brandon-bennett', 'https://eps.leeds.ac.uk/faculty-engineering-physical-sciences/staff/13223/harriet-bennett', 'https://eps.leeds.ac.uk/computing/staff/8810/dr-markus-billeter', 'https://eps.leeds.ac.uk/com

In [67]:
df = pd.DataFrame(columns=['name', 'text'])

count = 0
for i in url:
    response = requests.get(i)
    time.sleep(0.5)
    soup = BeautifulSoup(response.text, 'html.parser')
    cms = soup.find('div', class_='cms')
    profile = cms.text

    df.loc[count, 'text'] = profile
    count = count + 1

### Text cleaning

In [68]:
def capitalize_words(s):
    return s.title()

names = []

for i in url:
    name = i.rsplit('/', 1)[-1]  # Split the URL from the end ('/') and take the last part (-1)
    newName = name.replace('-', ' ')
    newName = newName.title()
    names.append(newName)  # Replace hyphens with spaces

In [69]:
df['name'] = pd.Series(names)

In [70]:
def expand_contractions(text):
    contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "I'd": "I would", "I'd've": "I would have","I'll": "I will",
                     "I'll've": "I will have","I'm": "I am","I've": "I have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have","you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

    contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))
    
    def replace(match):
        return contractions_dict[match.group(0)]
    
    return contractions_re.sub(replace, text)

In [71]:
# 定义一个函数来清洁文本
def clean_text(text):
    # 移除HTML标签
    no_html = re.sub('<.*?>', '', text)
    # 移除 'Publications:' 和它之后的所有文本
    no_publications = re.sub('Publications:.*', '', no_html, flags=re.S)
    # 移除 '\xa0'
    no_special_char = re.sub('\xa0', ' ', no_publications)
    # 移除 '\n'
    cleaned_text = re.sub('\n', ' ', no_special_char)
    # 移除连续的空白字符
    cleaned_text = re.sub(' +', ' ', cleaned_text)

    to_remove = ['Research projects', 'Profile', 'Research groups and institutes', 
                 'Current postgraduate researchers', 'Postgraduate research opportunities', 
                 'Projects', 'Research interests', 'Research groups and institutes', 
                 'Research Funding', 'Qualifications', 'Student education', 'Student feedback', 
                 'Current MSc Researchers', 'Current postgraduate researchers', 
                 'Postgraduate research opportunities', 'Research Grants', 
                 'Responsibilities', 'Professional memberships', 'Current postgraduate researchers','Any research projects']

    for phrase in to_remove:
        cleaned_text = cleaned_text.replace(phrase, '')

    return cleaned_text.strip()  # .strip() 移除字符串开头和结尾的空白字符

# 应用清洁函数到'text'列上
df['text'] = df['text'].apply(clean_text)
df['text'] = df['text'].apply(expand_contractions)

In [72]:
df.to_csv('original_data.csv', index=False)

### Text filtering and further cleaning

In [73]:
# 加载spaCy英文模型
nlp = spacy.load('en_core_web_sm')

def filter_words(text):
    # 使用spaCy进行词性标注
    doc = nlp(text)
    
    # 只保留名词、形容词、副词和专有名词
    filtered_sentence = [token.text for token in doc if token.pos_ in ('NOUN', 'ADJ', 'ADV', 'PROPN')]
    
    return filtered_sentence

def clean_text1(text):
    # 删除html标签
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text(separator=" ")

    # 删除Publications: 之后的文字，包括publications 
    text = re.split(r'publications', text, flags=re.I)[0]

    # 删除\xa0和\n等非法字符
    text = text.replace('\xa0', ' ').replace('\n', ' ').replace('\r', ' ')

    # 删除额外的空格
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def clean_text2(text):
    # 删除html标签
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text(separator=" ")

    # 删除Publications: 之后的文字，包括publications 
    text = re.split(r'publications', text, flags=re.I)[0]

    # 删除\xa0和\n等非法字符
    text = text.replace('\xa0', ' ').replace('\n', ' ').replace('\r', ' ')

    # 删除额外的空格
    text = re.sub(r'\s+', ' ', text).strip()

    # 删除主语、动词和连接词
    text = ' '.join(filter_words(text))

    return text

### Self-trained Word2Vec model for recommendation

In [74]:
def recommend_tutors(df, user_input):
    # 清理文本并进行分词
    df['text_cleaned'] = df['text'].apply(lambda x: nltk.word_tokenize(clean_text1(x).lower()))

    # 训练Word2Vec模型
    model = Word2Vec(sentences=df['text_cleaned'], vector_size=100, window=5, min_count=1, workers=4)

    # 定义函数将文本转换为向量
    def text_to_vector(word_list):
        # 使用Word2Vec模型将每个单词转换为向量，然后取平均
        word_vectors = np.array([model.wv[word] for word in word_list if word in model.wv.index_to_key])
        return word_vectors.mean(axis=0) if word_vectors.size != 0 else np.zeros(model.vector_size)

    # 将导师描述和用户输入转换为向量
    df['vector'] = df['text_cleaned'].apply(text_to_vector)
    user_vector = text_to_vector(nltk.word_tokenize(clean_text2(user_input).lower())) # 删除主语和动词

    # 计算相似度并推荐导师
    df['similarity'] = df['vector'].apply(lambda x: cosine_similarity([x], [user_vector])[0][0])
    recommended_tutors = df.sort_values('similarity', ascending=False)

    return recommended_tutors

input_text = "I like playing games and artificial intelligence"
recommended_tutors1 = recommend_tutors(df, input_text)

In [75]:
recommended_tutors1.head(5)

,name,text,text_cleaned,vector,similarity
7,Professor Eric Atwell,I teach and research in Artificial Intelligenc...,"[i, teach, and, research, in, artificial, inte...","[-0.1977614, 0.25478977, 0.0013315061, 0.00619...",0.999701
83,Claire Savy,EPSRC Centre for Doctoral Training Leeds Insti...,"[epsrc, centre, for, doctoral, training, leeds...","[-0.17421457, 0.22702046, 0.0025590896, 0.0086...",0.999700
47,Dr Yanlong Huang,I am a University Academic Fellow at the Schoo...,"[i, am, a, university, academic, fellow, at, t...","[-0.20359279, 0.26056474, 0.001008607, 0.00606...",0.999607
98,Professor Jie Xu,I am the leader for a Research Peak of Excelle...,"[i, am, the, leader, for, a, research, peak, o...","[-0.22879526, 0.2941806, 0.0007874857, 0.00639...",0.999581
65,Dr Zeinab Nezami,I have joined the School of Computing as a res...,"[i, have, joined, the, school, of, computing, ...","[-0.22105634, 0.28267562, 0.00018025635, 0.004...",0.999531


### Google word2vec model: GoogleNews-vectors-negative300

In [76]:
model_name = "word2vec-google-news-300"  # 模型名称
model_fasttext = api.load(model_name)

In [77]:
# 加载Google News Word2Vec模型
model = model_fasttext

def text_to_vector(word_list):
    # 使用Word2Vec模型将每个单词转换为向量，然后取平均
    word_vectors = np.array([model[word] for word in word_list if word in model])
    return word_vectors.mean(axis=0) if word_vectors.size != 0 else np.zeros(300)

def recommend_tutors(df, user_input):
    # 清理文本并进行分词
    df['text_cleaned'] = df['text'].apply(lambda x: nltk.word_tokenize(clean_text1(x).lower()))

    # 将导师描述和用户输入转换为向量
    df['vector'] = df['text_cleaned'].apply(text_to_vector)
    user_vector = text_to_vector(nltk.word_tokenize(clean_text2(user_input).lower()))
  
    # 计算相似度并推荐导师
    df['similarity'] = df['vector'].apply(lambda x: cosine_similarity([x], [user_vector])[0][0])
    recommended_tutors = df.sort_values('similarity', ascending=False)

    return recommended_tutors

input_text = "I like softerware engineering"
recommended_tutors2 = recommend_tutors(df, input_text)

In [78]:
recommended_tutors2.head(5)

,name,text,text_cleaned,vector,similarity
2,Dr Ban Al Jassani,Senior programmer with valuable experience of ...,"[senior, programmer, with, valuable, experienc...","[-0.010181222, -0.020750968, 0.01921869, 0.108...",0.565224
78,Dr Thomas Richardson,Real-time systems Safety-critical systems Embe...,"[real-time, systems, safety-critical, systems,...","[-0.0061060586, -0.020157496, 0.07227198, 0.13...",0.557450
89,Dr Mark Walkley,I began by studying for a BSc Engineering Math...,"[i, began, by, studying, for, a, bsc, engineer...","[-0.010378605, 0.008214832, 0.06386979, 0.1168...",0.545311
66,Dr Thai Thu Hien Nguyen,"2007 – 2008 MSc Chemical Engineering, Universi...","[2007, –, 2008, msc, chemical, engineering, ,,...","[-0.071736656, 0.0035597484, 0.062740326, 0.09...",0.541348
72,Dr Jonathan Pickering,Research Metallurgist at what is now part Qine...,"[research, metallurgist, at, what, is, now, pa...","[-0.008087158, -0.0153341545, 0.03571808, 0.06...",0.504995


### jaccard similarity

In [79]:
def jaccard_similarity(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return len(s1.intersection(s2)) / len(s1.union(s2))

def recommend_tutors(df, user_input):
    # 清理文本并进行分词
    df['words_set'] = df['text'].apply(lambda x: set(nltk.word_tokenize(clean_text1(x).lower())))

    # 将用户输入转换为词集合
    user_words_set = set(nltk.word_tokenize(clean_text1(user_input).lower()))

    # 计算Jaccard相似度并推荐导师
    df['similarity'] = df['words_set'].apply(lambda x: jaccard_similarity(x, user_words_set))
    recommended_tutors = df.sort_values('similarity', ascending=False)

    return recommended_tutors

input_text = "I like data mining and text analysis, and I am good at coding and I am also interested in artificial intelligence"
recommended_tutors3 = recommend_tutors(df, input_text)

In [80]:
recommended_tutors3.head(5)

,name,text,text_cleaned,vector,similarity,words_set
71,Dr Rafael Papallas,I am a Research Fellow in the School of Comput...,"[i, am, a, research, fellow, in, the, school, ...","[0.015335693, 0.023737794, -0.0051464844, 0.07...",0.171429,"{interest, control, manipulation, school, comp..."
94,Dr Lijun Wei,I am currently working on will be listed below...,"[i, am, currently, working, on, will, be, list...","[-0.030834056, 0.020737259, 0.033114962, 0.100...",0.153846,"{list, listed, search, allows, view, will, pro..."
6,Dr Abdulrahman Altahhan,"I have extesive experience in developing, lead...","[i, have, extesive, experience, in, developing...","[-0.032901, 0.0530971, 0.034158763, 0.08209403...",0.130435,"{leading, network, undergraduate, ai, research..."
5,Dr Mohammad Ammar Alsalka,MSc Coordinator Natural language processing\...,"[msc, coordinator, natural, language, processi...","[-0.019429266, 0.013454703, 0.030979978, 0.076...",0.107527,"{mining, listed, sarah, from, allows, phd, vie..."
17,Professor Andy Bulpitt,Head of School Andy is research interests are...,"[head, of, school, andy, is, research, interes...","[-0.029364828, 0.03062004, 0.043728434, 0.0967...",0.107143,"{scholarships, ;, list, fhea, allcock, listed,..."


Doc2Vec for input and 

In [81]:
def tag_docs(docs):
    tagged = [TaggedDocument(words=words, tags=[i]) for i, words in enumerate(docs)]
    return tagged

def train_doc2vec_model(tagged_docs):
    model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4)
    model.build_vocab(tagged_docs)
    model.train(tagged_docs, total_examples=model.corpus_count, epochs=model.epochs)
    return model

def vectorize_docs(model, tagged_docs):
    vectors = [model.infer_vector(doc.words, epochs=50) for doc in tagged_docs]
    return vectors

# 清理文本并进行分词
df['text_cleaned'] = df['text'].apply(lambda x: nltk.word_tokenize(clean_text1(x).lower()))

input_text = "I like data mining and text analysis"

# 将用户输入也转换为清理后的词列表
user_input = nltk.word_tokenize(clean_text1(input_text).lower())

# 为导师描述和用户输入生成TaggedDocument对象
tagged_tutor_descs = tag_docs(df['text_cleaned'])
tagged_user_input = TaggedDocument(words=user_input, tags=['user_input'])

# 训练Doc2Vec模型
model = train_doc2vec_model(tagged_tutor_descs)

# 将导师描述和用户输入转换为向量
df['vector'] = vectorize_docs(model, tagged_tutor_descs)
user_vector = model.infer_vector(user_input, epochs=50)

# 计算相似度并推荐导师
df['similarity'] = df['vector'].apply(lambda x: cosine_similarity([x], [user_vector])[0][0])
recommended_tutors4 = df.sort_values('similarity', ascending=False)

In [82]:
recommended_tutors4

,name,text,text_cleaned,vector,similarity,words_set
98,Professor Jie Xu,I am the leader for a Research Peak of Excelle...,"[i, am, the, leader, for, a, research, peak, o...","[0.023332037, -0.3478638, -1.0441376, 0.917687...",0.984497,"{chair, rolls-royce, from, director, phd, uk, ..."
6,Dr Abdulrahman Altahhan,"I have extesive experience in developing, lead...","[i, have, extesive, experience, in, developing...","[-0.07924924, -0.12616201, -0.4281729, 0.33309...",0.982795,"{leading, network, undergraduate, ai, research..."
47,Dr Yanlong Huang,I am a University Academic Fellow at the Schoo...,"[i, am, a, university, academic, fellow, at, t...","[-0.22143364, -0.44159117, -1.0921178, 1.13937...",0.981157,"{;, robots, experiment, enquiry, from, project..."
55,Dr Wiktoria Lawniczak,I am currently working on will be listed below...,"[i, am, currently, working, on, will, be, list...","[-0.09220795, -0.051815156, -0.30924588, 0.270...",0.969077,"{list, listed, search, allows, view, will, pro..."
25,Mr Minh Thang Dang,I am currently working on will be listed below...,"[i, am, currently, working, on, will, be, list...","[-0.08533053, -0.05351593, -0.30850932, 0.2595...",0.966700,"{list, listed, search, allows, view, will, pro..."
...,...,...,...,...,...,...
35,Charlotte Francis,,[],"[-0.0015568888, -0.0031317996, 0.0023990143, 0...",-0.040483,{}
44,Beth Hilditch,,[],"[-0.0007370472, -0.0022524213, 0.0048008556, 0...",-0.054683,{}
16,Ruksana Bukhari Bibi,,[],"[-0.002988563, 0.0022667097, -0.0019968182, 0....",-0.085164,{}
39,Sunjeev Ghir,,[],"[-0.003253906, -1.1675358e-05, 0.0040615727, -...",-0.089134,{}


### CountVectorizer

In [83]:
def recommend_tutors(df, user_input):
    # 清理文本
    df['text_cleaned'] = df['text'].apply(clean_text2) # 删除主语动词连接词，因为CountVectorizer只考虑词频，所以主语等会有很大影响
    
    # 使用CountVectorizer将导师描述转换为向量
    vectorizer = CountVectorizer(stop_words='english')
    vectors = vectorizer.fit_transform(df['text_cleaned'])
    
    # 使用相同的Vectorizer将用户输入转换为向量
    user_vector = vectorizer.transform([clean_text2(user_input)])
    
    # 计算相似度并推荐导师
    df['similarity'] = cosine_similarity(vectors, user_vector).flatten()
    recommended_tutors = df.sort_values('similarity', ascending=False)
    
    return recommended_tutors

input_text = "I like software engineering"
recommended_tutors5 = recommend_tutors(df, input_text)

In [84]:
recommended_tutors5

,name,text,text_cleaned,vector,similarity,words_set
72,Dr Jonathan Pickering,Research Metallurgist at what is now part Qine...,Research Metallurgist now part Retrained Compu...,"[-0.32746148, -0.14802234, -1.100363, 0.847594...",0.431934,"{;, listed, salford, provision, allows, based,..."
56,Mr Alejandro Lebrero,Alejandro Lebrero is a Research Software Engin...,Alejandro Lebrero Research Software Engineer C...,"[-0.2923123, -0.09745591, -1.0419152, 0.809626...",0.390360,"{researchers, tasks, multiple, cloud-based, ar..."
78,Dr Thomas Richardson,Real-time systems Safety-critical systems Embe...,Real time systems Safety critical systems Embe...,"[-0.21116793, -0.09511463, -0.54568845, 0.4695...",0.328798,"{list, listed, search, allows, object-oriented..."
89,Dr Mark Walkley,I began by studying for a BSc Engineering Math...,BSc Engineering Mathematics Civil Engineering ...,"[-0.6092309, 0.1214888, -0.7083839, 0.58423465...",0.324443,"{listed, waves, from, followed, allows, phd, v..."
66,Dr Thai Thu Hien Nguyen,"2007 – 2008 MSc Chemical Engineering, Universi...",MSc Chemical Engineering University Leeds UK P...,"[-0.19119263, -0.036772914, -0.6059019, 0.4196...",0.283473,"{2008, list, listed, –, certificate, search, a..."
...,...,...,...,...,...,...
39,Sunjeev Ghir,,,"[-0.003253906, -1.1675358e-05, 0.0040615727, -...",0.000000,{}
38,Joseph Ghaffari Motlagh,I am currently working on will be listed below...,currently below list research projects full li...,"[-0.123966135, -0.01998213, -0.26272616, 0.216...",0.000000,"{list, listed, search, allows, view, will, pro..."
35,Charlotte Francis,,,"[-0.0015568888, -0.0031317996, 0.0023990143, 0...",0.000000,{}
34,Dr Samson Fabiyi,I am currently working on will be listed below...,currently below list research projects full li...,"[-0.18570183, -0.029170252, -0.36319235, 0.298...",0.000000,"{list, listed, search, allows, phd, view, will..."


### TF-IDF

In [85]:
def recommend_tutors(df, user_input):
    # 清理文本
    df['text_cleaned'] = df['text'].apply(clean_text1)
    
    # 使用TfidfVectorizer将导师描述转换为向量
    vectorizer = TfidfVectorizer(stop_words='english')
    vectors = vectorizer.fit_transform(df['text_cleaned'])
    
    # 使用相同的Vectorizer将用户输入转换为向量
    user_vector = vectorizer.transform([clean_text2(user_input)])
    
    # 计算相似度并推荐导师
    df['similarity'] = cosine_similarity(vectors, user_vector).flatten()
    recommended_tutors = df.sort_values('similarity', ascending=False)
    
    return recommended_tutors

input_text = "I like software engineering"
recommended_tutors6 = recommend_tutors(df, input_text)

In [86]:
recommended_tutors6

,name,text,text_cleaned,vector,similarity,words_set
72,Dr Jonathan Pickering,Research Metallurgist at what is now part Qine...,Research Metallurgist at what is now part Qine...,"[-0.32746148, -0.14802234, -1.100363, 0.847594...",0.358621,"{;, listed, salford, provision, allows, based,..."
56,Mr Alejandro Lebrero,Alejandro Lebrero is a Research Software Engin...,Alejandro Lebrero is a Research Software Engin...,"[-0.2923123, -0.09745591, -1.0419152, 0.809626...",0.355159,"{researchers, tasks, multiple, cloud-based, ar..."
78,Dr Thomas Richardson,Real-time systems Safety-critical systems Embe...,Real-time systems Safety-critical systems Embe...,"[-0.21116793, -0.09511463, -0.54568845, 0.4695...",0.285168,"{list, listed, search, allows, object-oriented..."
10,Dr Amy Lowe,I am currently working on will be listed below...,I am currently working on will be listed below...,"[-0.16633962, -0.08422261, -0.45219773, 0.3750...",0.213017,"{list, project, listed, comp3631, search, allo..."
89,Dr Mark Walkley,I began by studying for a BSc Engineering Math...,I began by studying for a BSc Engineering Math...,"[-0.6092309, 0.1214888, -0.7083839, 0.58423465...",0.210432,"{listed, waves, from, followed, allows, phd, v..."
...,...,...,...,...,...,...
39,Sunjeev Ghir,,,"[-0.003253906, -1.1675358e-05, 0.0040615727, -...",0.000000,{}
38,Joseph Ghaffari Motlagh,I am currently working on will be listed below...,I am currently working on will be listed below...,"[-0.123966135, -0.01998213, -0.26272616, 0.216...",0.000000,"{list, listed, search, allows, view, will, pro..."
35,Charlotte Francis,,,"[-0.0015568888, -0.0031317996, 0.0023990143, 0...",0.000000,{}
34,Dr Samson Fabiyi,I am currently working on will be listed below...,I am currently working on will be listed below...,"[-0.18570183, -0.029170252, -0.36319235, 0.298...",0.000000,"{list, listed, search, allows, phd, view, will..."


#### OLD

In [87]:
# def preprocess_text(text):
#     """
#     对文本进行预处理
#     """
#     text = text.lower()  # 转换为小写字母
#     text = text.replace("'", "")  # 去除单引号
#     text = text.replace("-", "")  # 去除破折号
#     text = "".join([char for char in text if char.isalpha() or char.isspace()])  # 保留字母和空格
#     return text

# def tokenize_text(text):
#     """
#     将文本转换为tokens列表
#     """
#     tokens = text.split()
#     return tokens

# def plot_most_frequent_words(tokens, title):
#     """
#     绘制文本中出现频率最高的词汇柱状图
#     """
#     fdist = FreqDist(tokens)
#     words_df = pd.DataFrame({'word': list(fdist.keys()), 'count': list(fdist.values())})
#     # 选择出现频率最高的前20个词汇
#     d = words_df.nlargest(columns="count", n=10)
#     plt.figure(figsize=(16, 9))
#     sns.barplot(data=d, x="word", y="count")
#     plt.title(title)
#     plt.show()


# def plot_pearson_correlation_coefficient(X, data):
#     """
#     绘制各个文本之间的Pearson相关系数热力图
#     """
#     corr_matrix = np.zeros((len(data), len(data)))
#     for i, j in itertools.combinations(range(len(data)), 2):
#         corr, _ = pearsonr(X.toarray()[i], X.toarray()[j])
#         corr_matrix[i, j] = corr
#         corr_matrix[j, i] = corr
#     tick_labels = data['title']
#     sns.set(style="white")
#     mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
#     f, ax = plt.subplots(figsize=(10, 8))
#     cmap = sns.diverging_palette(230, 20, as_cmap=True)
#     sns.heatmap(corr_matrix, mask=mask, cmap=cmap, vmax=.3, center=0,
#                 square=True, linewidths=.5, cbar_kws={"shrink": .5},
#                 xticklabels=tick_labels, yticklabels=tick_labels,
#                 annot=True, fmt='.2f', annot_kws={'size': 10})
#     plt.title("Pearson Correlation Coefficient Matrix")
#     plt.show()


# def recommend_courses(sinput, df):
#     # 引入CountVectorizer库，用于文本向量化
#     vectorizer = CountVectorizer(tokenizer=tokenize_text, preprocessor=preprocess_text)

#     # 将data中的文本转换成向量X，每行代表一个课程的向量表示
#     X = vectorizer.fit_transform(df['text'])

#     # 将用户输入的文本转换成向量表示
#     student_input = vectorizer.transform(sinput.split())

#     # 计算用户输入的文本和所有课程向量之间的余弦相似度
#     cosine_sim = cosine_similarity(X, student_input)

#     # 为每个课程在data中增加一列'similarity'，表示该课程与用户输入的文本的相似度得分
#     df['similarity'] = cosine_sim.sum(1)

#     # 根据'similarity'得分进行降序排列，选择前3个得分最高
#     top_courses = df.sort_values('similarity', ascending=False).head(3)['Name']
#     df = df.sort_values('similarity', ascending=False)
#     #return top_courses.values
#     return df

In [88]:
# df['text'] = df['text'].apply(lambda x: re.sub(r'<.*?>', '', x)) # Remove HTML tags
# df['text'] = df['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x)) # Remove punctuation
# df['text'] = df['text'].apply(lambda x: x.lower()) # Convert to lowercase
# # Tokenize data
# nltk.download('punkt')
# df['tokens'] = df['text'].apply(lambda x: nltk.word_tokenize(x))

# # Remove stop words
# nltk.download('stopwords')
# stop_words = set(stopwords.words('english'))
# df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

# # Lemmatize data
# nltk.download('wordnet')
# lemmatizer = WordNetLemmatizer()
# df['tokens'] = df['tokens'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])


Supervisors which have more contents will get more recommendations.  

Supervisors with short contents have a limited number of keywords and do not easily get recommendations.  

There are some supervisors who often get to be at the top of the recommendation table because their contents are so long and therefore the number of keywords is also bigger than others, and no matter what you enter, they always take the top spot in the recommendations.  

(Sharib Ali and Noorhan Abbas always gets high similarity scores.)

### Text generation using Bart model

In [92]:
# device = torch.device("mps:0")
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [93]:
# Initialize the tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
modelB = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

# Function to generate summaries
def generate_summary_brat(text):
    inputs = tokenizer.encode_plus(text, truncation=True, max_length=1024, return_tensors='pt')
    input_ids = inputs['input_ids'].to(modelB.device)
    attention_mask = inputs['attention_mask'].to(modelB.device)
    summary_ids = modelB.generate(input_ids, attention_mask=attention_mask, num_beams=8, max_length=100, early_stopping=False) # max_length=100 is original set
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [94]:
df['summary'] = df['text'].apply(generate_summary_brat)

In [96]:
df.drop(['similarity'], axis=1, inplace=True)

In [97]:
df

,name,text,text_cleaned,vector,words_set,summary
0,Dr Noorhan Abbas,I teach and research in Artificial Intelligenc...,I teach and research in Artificial Intelligenc...,"[-0.7777705, -0.056486133, -1.0847702, 0.86731...","{mining, post-doctoral, skrynnyk, listed, whic...",I teach and research in Artificial Intelligenc...
1,Professor Isolde Adler,I am currently working on will be listed below...,I am currently working on will be listed below...,"[-0.08300763, -0.057043005, -0.26899123, 0.241...","{list, listed, search, allows, view, will, pro...",I am currently working on will be listed below...
2,Dr Ban Al Jassani,Senior programmer with valuable experience of ...,Senior programmer with valuable experience of ...,"[-0.3869004, 0.024613451, -0.8536577, 0.542717...","{listed, allows, phd, view, will, expertise, h...",Senior programmer with valuable experience of ...
3,Dr Bogdan Alecu,I am currently working on will be listed below...,I am currently working on will be listed below...,"[-0.11009211, -0.03453691, -0.27944672, 0.2354...","{list, listed, search, allows, view, will, pro...",I am currently working on will be listed below...
4,Dr Sharib Ali,I obtained my PhD in image analysis (awarded i...,I obtained my PhD in image analysis (awarded i...,"[-0.6224948, -0.41804323, -0.9384999, 1.153657...","{listed, gupta, train, lead, patients, previou...",I obtained my PhD in image analysis (awarded i...
...,...,...,...,...,...,...
96,Dr Samuel Wilson,Director of Student Education I am currently...,Director of Student Education I am currently w...,"[-0.17392215, 0.0025068407, -0.3190983, 0.2256...","{list, listed, student, search, allows, direct...",Director of Student Education I am currently...
97,Dr Xia,Computational Medicine,Computational Medicine,"[-0.014929377, 0.002028018, -0.07792133, 0.054...","{computational, medicine}",Computational Medicine
98,Professor Jie Xu,I am the leader for a Research Peak of Excelle...,I am the leader for a Research Peak of Excelle...,"[0.023332037, -0.3478638, -1.0441376, 0.917687...","{chair, rolls-royce, from, director, phd, uk, ...",I am the leader for a Research Peak of Excelle...
99,Dr Omer Yuval,The neurons that control living organisms are ...,The neurons that control living organisms are ...,"[-0.6351099, -0.124341965, -1.0145229, 0.97335...","{variety, listed, controlling, analogies, allo...",The neurons that control living organisms are ...


In [98]:
df.loc[0, 'summary']

'I teach and research in Artificial Intelligence, Data Science, Data Mining and Text Analytics, in the School of Computing at University of Leeds. I worked in the Edubots project which is an Erasmus+ Knowledge Alliance project that aims to improve results and raise attainment levels in European higher education institutions. This project focussed on exploring best practices and innovative use of chatbots, and creating a learning community of educators. I have experience in organising conferences and workshops. I organised two conferences for'

### TextRank generation

In [115]:
def generate_summary_textRank(x):
    model = Word2Vec(
        sentences=[word_tokenize(x)],
        vector_size=100, window=5, min_count=5, workers=2
    )

    sentences = sent_tokenize(x)
    sentences_feat = [
        np.array([model.wv[x] for x in word_tokenize(sentence.lower()) if x in model.wv]).mean(0)
        for sentence in sentences
    ]

    sentences_feat = np.array(sentences_feat)

    from sklearn.metrics.pairwise import cosine_similarity
    sim_mat = np.zeros([len(sentences), len(sentences)])
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(
                    sentences_feat[i].reshape(1, 100),
                    sentences_feat[j].reshape(1, 100)
                )[0, 0]

    nx_graph = nx.from_numpy_array(sim_mat)
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                nx_graph[i][j]['weight'] = sim_mat[i][j]
    
    scores = nx.pagerank(nx_graph)

    # 排序句子的得分，并获取前三个得分最高的句子
    sorted_scores = sorted(((score, idx) for idx, score in scores.items()), reverse=True)
    top_sentence_indices = [idx for score, idx in sorted_scores[:3]]

    # 返回得分最高的句子，并保持它们在原文本中的顺序
    return ' '.join(sentences[idx] for idx in sorted(top_sentence_indices))


In [116]:
generate_summary_textRank(df.loc[0, 'text'])

'It brought more interactions with students and the teacher. The teacher Noorhan is very responsible and always responsive. She’s the heart and soul of the module.'

In [117]:
df['summary_textRank'] = df['text'].apply(generate_summary_textRank)

RuntimeError: you must first build vocabulary before training the model

Comparison with Bart